In [2]:
import nibabel as nib
import numpy as np
import tensorflow as tf
import os
import configparser
from configparser import ConfigParser

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def getImageFiles(patient, trainingOrTesting):
  """
  Gets the directory paths in a dictionary of all relevant images
  trainingOrTesting = "training" or "testing"
  """
  fileDic = {}
  nonDataList = ['MANDATORY_CITATION.md', 'Info.cfg', patient + "_4d.nii.gz"]
  directory_path = '/content/drive/MyDrive/Superresolution/database/' + trainingOrTesting
  directory_files = os.listdir(directory_path)
  for patientFile in directory_files:
    fileDic[patientFile] = []
    if patientFile in nonDataList:  #filter out the non-image files
      continue
    directoryPatient = directory_path + "/" + str(patientFile)
    filesPatient = os.listdir(directoryPatient)
    for imageFile in filesPatient:
      if imageFile in nonDataList:  #filter out the non-image files
        continue
      directoryPatientImage = directoryPatient + "/" + str(imageFile)
      fileDic[patientFile].append(directoryPatientImage)
  return fileDic[patient]

def getInfoFiles(patient, trainingOrTesting):
  """
  Gets the directory paths in a dictionary of all relevant images
  trainingOrTesting = "training" or "testing"
  """
  fileDic = {}
  nonDataList = ['MANDATORY_CITATION.md', 'Info.cfg']
  directory_path = '/content/drive/MyDrive/Superresolution/database/' + trainingOrTesting
  directory_files = os.listdir(directory_path)
  for patientFile in directory_files:
    fileDic[patientFile] = []
    if patientFile in nonDataList:  #filter out the non-image files
      continue
    directoryPatient = directory_path + "/" + str(patientFile)
    filesPatient = os.listdir(directoryPatient)
    for imageFile in filesPatient:
      if imageFile != 'Info.cfg':  #filter out the non-image files
        continue
      directoryPatientImage = directoryPatient + "/" + str(imageFile)
      fileDic[patientFile].append(directoryPatientImage)
  return fileDic[patient][0]

def patientNamesTraining():
  L = []
  for i in range(1, 101):
      if i < 10:
          patient = "patient" + "00" + str(i)
      elif 10 <= i <= 99:
          patient = "patient" + "0" + str(i)
      else:
          patient = "patient" + str(i)
      L.append(patient)
  return L

def patientNamesTesting():
  return [("patient" + str(i)) for i in range(101, 151)]

def getTrainingData():
  arrayDic = {}
  for patient in patientNamesTraining():
    arrayDic[patient] = []
    fileDir = getImageFiles(patient, 'training')
    for patientDir in fileDir:
      img = nib.load(patientDir)
      arrayDic[patient].append(img.get_fdata())
  return arrayDic

def getTestingData():
  arrayDic = {}
  for patient in patientNamesTesting():
    arrayDic[patient] = []
    fileDir = getImageFiles(patient, 'testing')
    for patientDir in fileDir:
      img = nib.load(patientDir)
      arrayDic[patient].append(img.get_fdata())
  return arrayDic

def labelTraining():
  L = patientNamesTraining()
  patientDic = {}
  for patient in L:
    infopath = getInfoFiles(patient, 'training')
    parser = configparser.ConfigParser()
    with open(infopath) as stream:
        parser.read_string("[top]\n" + stream.read())
    details_dict = dict(parser.items("top"))
    patientDic[patient] = details_dict['group']
  return patientDic

def labelTesting():
  L = patientNamesTesting()
  patientDic = {}
  for patient in L:
    infopath = getInfoFiles(patient, 'testing')
    parser = configparser.ConfigParser()
    with open(infopath) as stream:
        parser.read_string("[top]\n" + stream.read())
    details_dict = dict(parser.items("top"))
    patientDic[patient] = details_dict['group']
  return patientDic

In [31]:
# print(getTrainingData()) #X_training: dicionary of testing data for patient and their images in a list of arrays of 4 pictures
# print(getTestingData()) #X_testing: dicionary of testing data for patient and their images in a list of arrays of 4 pictures
# print(labelTraining()) #Y_training: dictionary of labels for patient groups
# print(labelTesting()) #Y_testing: dictionary of labels for patient groups

### Pre-Processing Data

In [34]:
x_train = getTrainingData() #X_training: dicionary of testing data for patient and their images in a list of arrays of 4 pictures
x_test = getTestingData() #X_testing: dicionary of testing data for patient and their images in a list of arrays of 4 pictures
y_train = labelTraining() #Y_training: dictionary of labels for patient groups
y_test = labelTesting() #Y_testing: dictionary of labels for patient groups

print("x_train:", x_train)
print("x_test:", x_test)


Streaming output truncated to the last 5000 lines.

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])], 'patient125': [array([[[ 18.,  21.,  21., ...,  19.,  18.,  17.],
        [ 18.,  21.,  21., ...,  19.,  18.,  17.],
        [ 18.,  21.,  21., ...,  19.,  18.,  17.],
        ...,
        [ 18.,  21.,  21., ...,  19.,  18.,  17.],
        [ 18.,  21.,  21., ...,  19.,  18.,  17.],
        [ 18.,  21.,  21., ...,  19.,  18.,  17.]],

       [[ 18.,  21.,  21., ...,  19.,  18.,  17.],
        [ 18.,  21.,  21., ...,  19.,  18.,  17.],
        [ 18.,  21.,  21., ...,  19.,  18.,  17.],
        ...,
        [ 18.,  22.,  21., ...,  73.,  44.,  19.],
        [ 18.,  21.,  21., ...,  51.,  52.,  26.],
        [ 18.,  21.,  21., ...,  38.,  31.,  22.]],

       [[ 18.,  21.,  21., ...,  19.,  1

In [35]:
print("y_train:", y_train)
print("y_test:", y_test)

y_train: {'patient001': 'DCM', 'patient002': 'DCM', 'patient003': 'DCM', 'patient004': 'DCM', 'patient005': 'DCM', 'patient006': 'DCM', 'patient007': 'DCM', 'patient008': 'DCM', 'patient009': 'DCM', 'patient010': 'DCM', 'patient011': 'DCM', 'patient012': 'DCM', 'patient013': 'DCM', 'patient014': 'DCM', 'patient015': 'DCM', 'patient016': 'DCM', 'patient017': 'DCM', 'patient018': 'DCM', 'patient019': 'DCM', 'patient020': 'DCM', 'patient021': 'HCM', 'patient022': 'HCM', 'patient023': 'HCM', 'patient024': 'HCM', 'patient025': 'HCM', 'patient026': 'HCM', 'patient027': 'HCM', 'patient028': 'HCM', 'patient029': 'HCM', 'patient030': 'HCM', 'patient031': 'HCM', 'patient032': 'HCM', 'patient033': 'HCM', 'patient034': 'HCM', 'patient035': 'HCM', 'patient036': 'HCM', 'patient037': 'HCM', 'patient038': 'HCM', 'patient039': 'HCM', 'patient040': 'HCM', 'patient041': 'MINF', 'patient042': 'MINF', 'patient043': 'MINF', 'patient044': 'MINF', 'patient045': 'MINF', 'patient046': 'MINF', 'patient047': 'MIN

In [22]:
dim1_train, dim2_train, dim3_train = x_train['patient001'][0].shape
dim1_test, dim2_test, dim3_test = x_test['patient101'][0].shape

# print(dim1_train, dim2_train, dim3_train)
# print(dim1_test, dim2_test, dim3_test)

# print(x_train.keys())
for patient in x_train.keys():
  dim1, dim2, dim3 = x_train[patient][0].shape
  if dim1 < dim1_train:
    dim1_train = dim1

  if dim2 < dim2_train:
    dim2_train = dim2

  if dim3 < dim3_train:
    dim3_train = dim3

for patient in x_test.keys():
  dim1, dim2, dim3 = x_test[patient][0].shape
  if dim1 < dim1_test:
    dim1_test = dim1

  if dim2 < dim2_test:
    dim2_test = dim2

  if dim3 < dim3_test:
    dim3_test = dim3


# print(dim1_train, dim2_train, dim3_train)
# print(dim1_test, dim2_test, dim3_test)

216 256 10
6 162 6
154 154 6
6 162 6


In [30]:
# for patient in x_test.keys():
#   for i in range(4):
#     print(x_test[patient])
#     x_test[patient][i] = x_test[patient][i][0:dim1_test]

#     for j in range(len(x_test[patient][i])):
#       x_test[patient][i][j] = x_test[patient][i][j][0:dim2_test]

#       for k in range(len(x_test[patient][i][j])):
#         x_test[patient][i][j][k] = x_test[patient][i][j][k][0:dim3_test]


#     # x_test[patient][i][1] = x_test[patient][i][1][0:dim2_test]
#     # x_test[patient][i][0] = x_test[patient][i][0][0:dim3_test]



# for patient in x_train.keys():
#   for i in range(4):
#     x_train[patient][i] = x_train[patient][i][0:dim1_train, 0:dim2_train, 0:dim3_train]



[array([0., 0., 0., 0., 0., 0.]), array([[[ 19.,  20.,  18.,  18.,  18.,  17.],
        [ 19.,  20.,  18.,  18.,  18.,  17.],
        [ 19.,  20.,  18.,  18.,  18.,  17.],
        ...,
        [ 19.,  20.,  18.,  18.,  18.,  17.],
        [ 19.,  20.,  18.,  18.,  18.,  17.],
        [ 19.,  20.,  18.,  18.,  18.,  17.]],

       [[ 19.,  20.,  18.,  18.,  18.,  17.],
        [ 19.,  20.,  18.,  18.,  18.,  17.],
        [ 26.,  30.,  18.,  18.,  18.,  17.],
        ...,
        [ 19.,  20.,  18.,  18.,  18.,  17.],
        [ 19.,  20.,  18.,  18.,  18.,  17.],
        [ 19.,  20.,  18.,  18.,  18.,  17.]],

       [[ 19.,  20.,  18.,  18.,  18.,  17.],
        [ 19.,  20.,  18.,  18.,  18.,  17.],
        [ 26.,  29.,  18.,  18.,  18.,  17.],
        ...,
        [ 19.,  20.,  18.,  18.,  18.,  17.],
        [ 19.,  20.,  18.,  18.,  18.,  17.],
        [ 19.,  20.,  18.,  18.,  18.,  17.]],

       [[ 19.,  20.,  18.,  18.,  18.,  17.],
        [ 19.,  20.,  18.,  18.,  18.,  17.],
 

IndexError: invalid index to scalar variable.

In [24]:
# print(x_train[patient][0])

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [23]:
# for patient in x_test.keys():
#   for i in range(4):
#     print(x_test[patient][i].shape)

# for patient in x_train.keys():
#   for i in range(4):
#     print(x_train[patient][i].shape)

(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 162, 6)
(6, 

import new data
reshape images
split the training data into training and validation data
standardize data
convert the dense representation of the classes to one-hot encoding